In [1]:
import pandas as pd

### 2015 source file spec

In [2]:
# Define the file path
file_path = r'C:\Users\domin\Documents\GitHub\xai_budgeting\data\raw\kdkf_2015_raw.xlsx'

# Load the Excel file to get sheet names
xls = pd.ExcelFile(file_path)

# Define the years
previous_year = 2014
start_year = 2015
target_year = 2016

# Set the pandas option to display floating point numbers using the decimal format
pd.options.display.float_format = '{:.5f}'.format

### Master Loop

In [3]:
# Initialize an empty DataFrame for the master data
master_2015 = pd.DataFrame()

# Loop over each sheet in the Excel file
for sheet_name in xls.sheet_names:
    # Check if the sheet name starts with "HRM2_KT_"
    if sheet_name.endswith("_HRM2"):
        # Load the sheet into a DataFrame, skipping the first three rows
        df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=0)

        # Merge the first row with the column headers
        df.columns = df.iloc[0, :2].tolist() + (df.columns[2:]).tolist()
        df = df.iloc[1:].reset_index(drop=True)

        # Update Headers of the df
        rename_dict = {
            'Rechnung': f'Rechnung {previous_year}',
            'Budget': f'Budget {start_year}',
            'Rechnung.1': f'Rechnung {start_year}',
            'Budget.1': f'Budget {target_year}'
        }
        df.rename(columns=rename_dict, inplace=True)

        # Rename the first column to 'HRM 2' and third column to "in 1000 CHF"
        df.columns.values[0] = 'HRM2-ID'
        df.columns.values[2] = 'HRM2-Bezeichnung'

        # Delete unecessary columns
        df.drop(df.columns[1], axis=1, inplace=True)
             
        # Add a new column with the DataFrame name (sheet name)
        df['Source'] = sheet_name
        df['Kanton'] = df['Source'].str.split('_HRM2').str[0]
        df['Year'] = 2015

        # Use pd.concat to add the DataFrame to the master DataFrame
        master_2015 = pd.concat([master_2015, df], ignore_index=True)

# Display the first few rows of the master DataFrame
master_2015.head()

,HRM2-ID,HRM2-Bezeichnung,Rechnung 2014,Budget 2015,Rechnung 2015,Budget 2016,Unnamed: 7,Source,Kanton,Year,Compte,Compte.1
0,ERFOLGSRECHNUNG,NaN,NaN,def.,NaN,NaN,NaN,ZH_HRM2,ZH,2015,NaN,NaN
1,30,Personalaufwand,4958624.94034,5058273.57900,5100269.19577,5164956.48600,NaN,ZH_HRM2,ZH,2015,NaN,NaN
2,31,Sach- und übriger Betriebsaufwand,2943690.29170,2976696.62825,2943087.05891,3012788.30700,NaN,ZH_HRM2,ZH,2015,NaN,NaN
3,davon 314,baulicher und betrieblicher Unterhalt,214002.93191,209706.10000,218499.45497,207089.30000,NaN,ZH_HRM2,ZH,2015,NaN,NaN
4,davon 3180,Wertberichtigungen auf Forderungen,-3425.25279,897,-7985.47879,2658.20000,NaN,ZH_HRM2,ZH,2015,NaN,NaN


### Translating FR columns to DE

In [9]:
# Display rows where the column 'Kanton' is 'HRM2_KT_VD'
filtered_rows = master_2015[master_2015['Kanton'] == 'VD']

# Display the filtered rows
filtered_rows.head()

,HRM2-ID,HRM2-Bezeichnung,Rechnung 2014,Budget 2015,Rechnung 2015,Budget 2016,Source,Kanton,Year,Compte,Compte.1
3323,30,Charges de personnel,2463959.26779,2407976.70000,2316101.87585,2363614.70000,VD_HRM2,VD,2015,2463959.26779,2316101.87585
3324,31,Charges de biens et services et autres charges...,872251.11242,833525.50000,758905.26863,715635.30000,VD_HRM2,VD,2015,872251.11242,758905.26863
3327,330,Immobilisations corporelles du PA,174496.12907,152011.90000,123303.09109,169751.20000,VD_HRM2,VD,2015,174496.12907,123303.09109
3328,332,Amortissements des immobilisations incorporelles,15429.93630,25548.90000,59936.74607,27616.90000,VD_HRM2,VD,2015,15429.93630,59936.74607
3329,339,Remboursement du découvert du bilan,0.00000,0.00000,0.00000,0.00000,VD_HRM2,VD,2015,0.00000,0.00000


In [5]:
# 1. Update 'Rechnung 2014' based on 'Compte'
master_2015['Rechnung 2014'] = master_2015.apply(
    lambda row: row['Compte'] if pd.isna(row['Rechnung 2014']) and not pd.isna(row['Compte']) else row['Rechnung 2014'],
    axis=1
)

# 2. Update 'Rechnung 2015' based on 'Compte.1'
master_2015['Rechnung 2015'] = master_2015.apply(
    lambda row: row['Compte.1'] if pd.isna(row['Rechnung 2015']) and not pd.isna(row['Compte.1']) else row['Rechnung 2015'],
    axis=1
)

# Drop the column named 'Unnamed: 7'
master_2015.drop('Unnamed: 7', axis=1, inplace=True)

# Display the first few rows of the updated DataFrame
master_2015.head()


,HRM2-ID,HRM2-Bezeichnung,Rechnung 2014,Budget 2015,Rechnung 2015,Budget 2016,Source,Kanton,Year,Compte,Compte.1
0,ERFOLGSRECHNUNG,NaN,NaN,def.,NaN,NaN,ZH_HRM2,ZH,2015,NaN,NaN
1,30,Personalaufwand,4958624.94034,5058273.57900,5100269.19577,5164956.48600,ZH_HRM2,ZH,2015,NaN,NaN
2,31,Sach- und übriger Betriebsaufwand,2943690.29170,2976696.62825,2943087.05891,3012788.30700,ZH_HRM2,ZH,2015,NaN,NaN
3,davon 314,baulicher und betrieblicher Unterhalt,214002.93191,209706.10000,218499.45497,207089.30000,ZH_HRM2,ZH,2015,NaN,NaN
4,davon 3180,Wertberichtigungen auf Forderungen,-3425.25279,897,-7985.47879,2658.20000,ZH_HRM2,ZH,2015,NaN,NaN


In [6]:
# Display rows where the column 'Kanton' is 'VD_HRM2'
filtered_rows = master_2015[master_2015['Kanton'] == 'VD_HRM2']

# Display the filtered rows
filtered_rows.head()

,HRM2-ID,HRM2-Bezeichnung,Rechnung 2014,Budget 2015,Rechnung 2015,Budget 2016,Source,Kanton,Year,Compte,Compte.1


### Before saving the data, we drop non-numeric "HRM2-IDs", because they would cause issues with subsequent automated analysis...

In [7]:
# Drop rows where 'HRM-ID' values are non-numeric
master_2015 = master_2015[pd.to_numeric(master_2015['HRM2-ID'], errors='coerce').notna()]

# Display the first few rows of the updated DataFrame
len(master_2015)

2184

In [8]:
# Define the path where the CSV file will be saved
output_path = r'C:\Users\domin\Documents\GitHub\xai_budgeting\data\ready\kdkf_2015_data.csv'

# Save the DataFrame to a CSV file
master_2015.to_csv(output_path, index=False)
